In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# specify substep parameters for interactive run
# this cell will be replaced during job run with the parameters from json within params subfolder
substep_params={   
    "min_score" : 0.05
}

In [ ]:
# load pipeline and step parameters - do not edit
from sinara.substep import get_pipeline_params, get_step_params
pipeline_params = get_pipeline_params(pprint=True)
step_params = get_step_params(pprint=True)

In [ ]:
# define substep interface
from sinara.substep import NotebookSubstep, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params)

substep.interface(
    inputs =    
    [ 
      { STEP_NAME: "data_prep", ENTITY_NAME: "coco_test_dataset"}, # dataset for detector testing from data_prep step
      { STEP_NAME: "model_pack", ENTITY_NAME: "bento_service"}  # stored BentoService from model_pack
    ],
    tmp_entities =
    [
        { ENTITY_NAME: "obj_detect_inference_files" }
    ],
    tmp_outputs =
    [
        { ENTITY_NAME: "coco_test_dataset" },  # ground-true test dataset files for use in next substep
        { ENTITY_NAME: "coco_inference_result_dataset" } # predicted test dataset files for use in next substep
    ],
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

In [ ]:
# run spark
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

### Loading a trained model from bento_service
(weights, configs)

#### Get and load bentoservice

In [ ]:
import os.path as osp

In [ ]:
from sinara.bentoml import load_bentoservice

# read trained model
print('read trained model')
inputs_model_pack = substep.inputs(step_name = "model_pack")
bento_service = load_bentoservice(inputs_model_pack.bento_service)

#### Unpacking and saving artifacts from bentoservice

In [ ]:
# Save binary model weights and config to tmp_entities.obj_detect_inference_files
from sinara.bentoml import save_bentoartifact_to_tmp

tmp_entities = substep.tmp_entities()
tmp_outputs = substep.tmp_outputs()

#model_name = bento_service.artifacts["model_name"].get()
model_path = osp.join(tmp_entities.obj_detect_inference_files, f"latest.pth")
cfg_path = osp.join(tmp_entities.obj_detect_inference_files, f"last_cfg.py")

save_bentoartifact_to_tmp(bento_service, artifact_name="weigths_pth", artifact_file_path=model_path)
save_bentoartifact_to_tmp(bento_service, artifact_name="mmengine_cfg", artifact_file_path=cfg_path)

### Loading test datasets (from step data_prep)

In [ ]:
from sinara.archive import SinaraArchive
import json

# copy test data and dataset_config from previos step (data_prep) to tmp
inputs_data_prep = substep.inputs(step_name = "data_prep")

archive = SinaraArchive(spark)
archive.unpack_files_from_store_to_tmp(store_path=inputs_data_prep.coco_test_dataset, tmp_entity_dir=tmp_outputs.coco_test_dataset)

# reading ground-true test dataset markup 
with open(osp.join(tmp_outputs.coco_test_dataset, "test_coco_annotations.json")) as f_id:
    test_coco_dataset = json.load(f_id)
    
print('Annotations count in test_coco_dataset:', len(test_coco_dataset['annotations']))
print('Keys count in test_coco_dataset:', list(test_coco_dataset))

### Inference model

#### Initializing modules from mmdetection, mmcv

In [ ]:
from mmdet.apis import init_detector, inference_detector

#### Initializing obj detector

In [ ]:
# build the obj detector from a config file and a checkpoint file
obj_detector = init_detector(osp.join(tmp_entities.obj_detect_inference_files, 'last_cfg.py'), 
                      osp.join(tmp_entities.obj_detect_inference_files, 'latest.pth'), 
                      device='cpu')

#### Run obj detector on a test dataset

##### Run obj detector on a coco_test_dataset and prepere eval_coco_dataset.json for eval step

In [ ]:
from tqdm import tqdm
import cv2

from utils.coco import convert_inference_results_to_coco
from utils.coco import dump_coco_json

eval_coco_dataset = dict(test_coco_dataset, **{"annotations" : []})
for coco_image in tqdm(test_coco_dataset['images'], desc='process predict'):
    image_file_path = osp.join(tmp_outputs.coco_test_dataset, coco_image['file_name'])
    image_height, image_width, channels = cv2.imread(image_file_path).shape
    out = inference_detector(obj_detector, [image_file_path])
    coco_out  = []
    for batch_i, out_img in enumerate(out):
        pred_instances = out_img.cpu().numpy().pred_instances
        coco_out.append(convert_inference_results_to_coco(pred_instances, coco_image, image_width, image_height))
        eval_coco_dataset["annotations"] += coco_out[0].get("annotations", [])
        
last_image_file_path = image_file_path

print('test_coco_dataset annotations size:', len(test_coco_dataset['annotations']))
print('eval_coco_dataset annotations size:', len(eval_coco_dataset['annotations']))

eval_coco_dataset['info']['iouType'] = 'bbox'
for ann in eval_coco_dataset['annotations']:
    ann['area'] = ann['bbox'][2] * ann['bbox'][3]

eval_coco_dataset_path =  osp.join(tmp_outputs.coco_inference_result_dataset, 'eval_coco_dataset.json')
dump_coco_json(eval_coco_dataset_path, eval_coco_dataset)
print(f"eval_coco_torch = {eval_coco_dataset_path}")

### Preview of object detection on an image

#### Visualize last predict to last image from coco_test_dataset

In [ ]:
import mmcv
from utils import visualizer_utils

# get visualizer
visualizer = visualizer_utils.build_visualizer()
# the dataset_meta is loaded from the checkpoint and then pass to the model in init_detector
visualizer.dataset_meta = obj_detector.dataset_meta
# show the results
image = mmcv.imread(last_image_file_path, channel_order='rgb')
visualizer.add_datasample(
    'new_result',
    image,
    data_sample= out[0],
    draw_gt=False,
    wait_time=0,
    out_file=None,
    pred_score_thr=0.01
)
visualizer.show()

In [ ]:
# stop spark
SinaraSpark.stop_session()